In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [2]:
def GetSoup(address):
    try:
        r = requests.get(address)
        if r.status_code != 200:
            print ("address can NOT connect = {address}".format(address= address))
            return False
        else:
            res = r.text
            soup = BeautifulSoup(res, "html.parser")
            return soup
    except Exception as err:
        errStr = 'Exception in GetSoup function ' + str(err)
        print (errStr)

In [9]:
def GetBookData(soup, main_address):
    try:
        ret = []
        for div_data in soup.findAll("div", {"class": "goodsImgW"}):
            sub_con = div_data.find("a")["href"]
            sub_address = main_address + sub_con
            info = GetDetailBookInfo(sub_address)
            #print (info)
            ret.append(info)
        return ret
    except Exception as err:
        errStr = 'Exception in GetBookData function ' + str(err)
        print (errStr)

In [10]:
import datetime

def GetDetailBookInfo(address):
    try:
        book_info = {"title":"", "author" :[], "publisher" : "", "date" :"", "price":0} 
        soup = GetSoup(address)
        if soup:
            #책 이름
            book_info["title"] = soup.find("meta")["content"].strip()
            
            # 저자
            for p_data in soup.find_all('a', attrs ={"href": re.compile(r"Result\?domain\=ALL\&author_yn")}):
                for child in p_data.descendants:
                    book_info["author"].append(child)
            # 출판사
            for p_data in soup.find_all('a', attrs ={"href": re.compile(r"Result\?domain\=ALL\&company_yn")}):
                for child in p_data.descendants:
                    book_info["publisher"] = child            
            # 출간일
            dd_data = soup.find("dd", attrs ={"class" : "pdDate"})
            if dd_data:
                #print (dd_data)
                for p_data in dd_data.find_all("p"):
                    #print (p_data)
                    p = re.compile(r"([\d]{4})년 ([\d]{2})월 ([\d]{2})일")
                    m = p.search(p_data.get_text())
                    if m:
                        book_info["date"] = datetime.datetime.strptime(m.group(), "%Y년 %m월 %d일").date()
                        break
            # 가격 (정가 기준)
            tr_data = soup.find("div",{"class": "tbGoodsInfo firstTb"}).find("tbody").find("tr")
                #print (tr_data)
            if tr_data:
                for th_data in tr_data.find_all("th", attrs={"scope" : "row"}):
                    for child in th_data.descendants:
                        if child == '정가':
                            for price in tr_data.find("td").descendants:
                                replace_dict = {r'\s+': "", ",": "", "원": ""}
                                pattern = re.compile('|'.join(replace_dict.keys()))
                                book_info["price"] = int(re.sub(pattern, '', price))
                                break
        return book_info
    except Exception as err:
        errStr = 'Exception in GetDetailBookInfo function ' + str(err)
        print (errStr)
        

In [11]:
address = "http://www.yes24.com/24/Category/NewProductList/001?PageNumber={page}"
main_address = "http://www.yes24.com"
for i in range(1,2): # page는 1부터 40까지
    page_address= address.format(page=i)
    soup = GetSoup(address)
    if soup:
        ret = GetBookData(soup, main_address)
    print (ret)
    #html = HTML()       
            

[{'date': datetime.date(2016, 12, 28), 'author': ['Shinjiro'], 'publisher': '조은세상', 'title': '[예약판매] TABOO TATTOO 타부 타투 3', 'price': 5000}, {'date': datetime.date(2016, 12, 28), 'author': ['Tanaka Strike'], 'publisher': '조은세상', 'title': '[예약판매] SERVAMP 서뱀프 7', 'price': 5000}, {'date': datetime.date(2017, 1, 2), 'author': ['묘조 기요코', '이민희'], 'publisher': '교유서가', 'title': '카프카답지 않은 카프카', 'price': 18000}, {'date': datetime.date(2017, 2, 10), 'author': ['이영수'], 'publisher': '서울고시각', 'title': '2017 지적직 지적전산학개론 단원별 기출문제집', 'price': 18000}, {'date': datetime.date(2017, 2, 10), 'author': ['이기영'], 'publisher': '서울고시각', 'title': '2017 A+ 국제무역사 1급', 'price': 32000}, {'date': datetime.date(2016, 12, 20), 'author': ['강지혜', '조승연'], 'publisher': '상상의집', 'title': '오 마이 갓! 어쩌다 사춘기 3', 'price': 10800}, {'date': datetime.date(2017, 1, 10), 'author': ['정종연'], 'publisher': '성안당', 'title': '2017 합격비법 6 전기설비기술기준 및 판단기준', 'price': 22000}, {'date': datetime.date(2017, 1, 10), 'author': ['전수기'], 'publisher': '성안